In [64]:
# merging datasets
import pandas as pd
ptv = pd.read_parquet('../data/curated/final_train_hospital_cbd_dist_data')
shop = pd.read_csv('../data/curated/agg_shopping_centre.csv')
household = pd.read_csv('../data/curated/dwellings_household.csv')
recreation = pd.read_csv('../data/curated/recreation_cleaned.csv')
population = pd.read_csv('../data/curated/total_population.csv')
park_domain = pd.read_parquet('../data/curated/parkres/parkres_domain.parquet')
school = pd.read_csv('../data/curated/house_independent_school.csv')

In [65]:
columns_to_drop = [
    'price', 'address', 'property_type', 'latitude', 'longitude',
    'Beds', 'Baths', 'Parking', 'bond', 'extracted_price', 'geometry',
    'index_right', 'sa2_code', 'sa2_name', 'chg_flag', 'chg_lbl',
    'sa3_code', 'sa3_name', 'sa4_code', 'sa4_name', 'gcc_code', 'gcc_name',
    'ste_code', 'ste_name', 'aus_code', 'aus_name', 'areasqkm', 'loci_uri',
    'geometry_proj', 'Unnamed: 0'
]

# Dropping the columns from the DataFrame
school = school.drop(columns=columns_to_drop)

In [66]:
joint = pd.merge(ptv, shop, on='sa2_code', how='left')
joint = joint.merge(household, left_on='sa2_code', right_on='SA2 code', how='left')
joint = joint.merge(recreation, on='sa2_code', how='left')
joint = joint.merge(population, on='sa2_code', how='left')
joint = joint.merge(park_domain, on='url', how='left')
joint = joint.merge(school, on='url', how='left')

In [67]:
joint = joint.drop(columns=['sa2_name_y', 'sa2_name_x'])

In [68]:
joint.columns

Index(['url', 'price', 'address_x', 'property_type', 'latitude', 'longitude',
       'Beds', 'Baths', 'Parking', 'bond', 'extracted_price_x', 'geometry',
       'index_right', 'sa2_code_x', 'chg_flag', 'chg_lbl', 'sa3_code',
       'sa3_name', 'sa4_code', 'sa4_name', 'gcc_code', 'gcc_name', 'ste_code',
       'ste_name', 'aus_code', 'aus_name', 'areasqkm', 'loci_uri',
       'geometry_proj', 'suburb', 'property_id', 'nearest_station_id',
       'train_station_distance_km', 'cbd_distance_km', 'nearest_hospital_id',
       'nearest_hospital_distance', 'mean_stores', 'total_stores',
       'count_shopping_centres', 'SA2 code', 'Region', 'ERP_2021', 'ERP_2026',
       'ERP_2031', 'ERP_2036', 'PNPD_2021', 'PNPD_2026', 'PNPD_2031',
       'PNPD_2036', 'POPD_2021', 'POPD_2026', 'POPD_2031', 'POPD_2036',
       'OPD_2021', 'OPD_2026', 'OPD_2031', 'OPD_2036', 'HHS_2021', 'HHS_2026',
       'HHS_2031', 'HHS_2036', 'SPD_2021', 'SPD_2026', 'SPD_2031', 'SPD_2036',
       'OCC_2021', 'OCC_2026', 'OC

In [69]:
print(len(joint))

9565


In [70]:
joint.to_parquet('../data/curated/features_domain.parquet')